In [1]:
from sklearn import cross_validation
from scikits.statsmodels.tools import categorical
from sklearn import linear_model
import time
import csv

In [2]:
execfile('ML_Challenge_data_preprocessing.py')

('Generated X with shape ', (347698, 1000))
('Generated y with shape ', (347698,))


In [3]:
trigrams = []
with open('possible_trigrams.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        trigrams = row

trigrams.sort()
trigrams_dict = {key: i for i, key in enumerate(trigrams)}
print trigrams_dict

{'ACC': 5, 'ATG': 14, 'AAG': 2, 'AAA': 0, 'ATC': 13, 'AAC': 1, 'ATA': 12, 'AGG': 10, 'CCT': 23, 'CTC': 29, 'AGC': 9, 'ACA': 4, 'AGA': 8, 'CAT': 19, 'AAT': 3, 'ATT': 15, 'CTG': 30, 'CTA': 28, 'ACT': 7, 'CAC': 17, 'ACG': 6, 'CAA': 16, 'AGT': 11, 'CCA': 20, 'CCG': 22, 'CCC': 21, 'TAT': 51, 'GGT': 43, 'TGT': 59, 'CGA': 24, 'CAG': 18, 'CGC': 25, 'GAT': 35, 'CGG': 26, 'CTT': 31, 'TGC': 57, 'GGG': 42, 'TAG': 50, 'GGA': 40, 'TAA': 48, 'GGC': 41, 'TAC': 49, 'GAG': 34, 'TCG': 54, 'TTA': 60, 'TTT': 63, 'GAC': 33, 'CGT': 27, 'GAA': 32, 'TCA': 52, 'GCA': 36, 'GTA': 44, 'GCC': 37, 'GTC': 45, 'GCG': 38, 'GTG': 46, 'TTC': 61, 'GTT': 47, 'GCT': 39, 'TGA': 56, 'TTG': 62, 'TCC': 53, 'TGG': 58, 'TCT': 55}


In [4]:
import time
execfile('ngram_features.py')

sample_size = 10000#X.shape[0])
start = time.clock()
newX = np.zeros((sample_size,len(trigrams_dict)))

for i in range(sample_size):
    newX[i,:] = ngram_features(X[i,:], 3, trigrams_dict)
finish = time.clock()
print 'time spent creating new features for {0} observations: {1}'.format(sample_size, finish - start)
# will take about 11 minutes for the whole data set
print newX.shape
print newX[0,:]

time spent creating new features for 10000 observations: 19.959651
(10000, 64)
[  1.   5.   4.   5.   3.  16.  12.   0.  10.  20.  27.   2.   0.   3.   4.
   7.   3.   7.  22.   4.  12.  68.  53.  27.  12.  58.  49.   8.   2.  15.
  35.   5.   7.  16.  30.   5.  16.  62.  51.  26.  30.  51.  59.  10.   4.
  12.   7.   2.   4.   3.   3.   0.   5.  14.  11.   4.   6.  27.  14.   5.
   4.   4.   6.   1.]


In [18]:
test_size = 0.1
y_sample = y[0:sample_size]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    newX, y_sample, test_size=test_size, random_state=0)
print X_train.shape
print y_train.shape

(9000, 64)
(9000,)


In [20]:
import time

start_time = time.clock()
logreg = linear_model.LogisticRegression(penalty='l1', fit_intercept = False, C=1e5)
logreg.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

time spent to train 0.9: 16.239782 seconds.


In [21]:
logreg.score(X_test, y_test)

0.89600000000000002

In [16]:
from sklearn.svm import SVC

start_time = time.clock()
# linear seems best so far achieving up to 0.89
clf = SVC(kernel = 'linear')
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
time spent to train 0.1: 3.443618 seconds.


In [17]:
clf.score(X_test, y_test)

0.88811111111111107

In [26]:
from sklearn.tree import DecisionTreeClassifier

start_time = time.clock()
clf = DecisionTreeClassifier(max_depth=10)
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')
time spent to train 0.9: 0.197333 seconds.


In [27]:
clf.score(X_test, y_test)

0.88100000000000001

In [36]:
from sklearn.ensemble import AdaBoostClassifier

start_time = time.clock()
clf = AdaBoostClassifier(n_estimators=200, learning_rate=1)
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
          n_estimators=200, random_state=None)
time spent to train 0.9: 4.315694 seconds.


In [37]:
clf.score(X_test, y_test)

0.89400000000000002

In [38]:
from sklearn.ensemble import RandomForestClassifier

start_time = time.clock()
clf = RandomForestClassifier(n_estimators=200, criterion='entropy', max_features=16)
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=16, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
time spent to train 0.9: 6.833194 seconds.


In [39]:
clf.score(X_test, y_test)

0.90600000000000003

In [40]:
from sklearn.naive_bayes import GaussianNB

start_time = time.clock()
clf = GaussianNB()
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

GaussianNB()
time spent to train 0.9: 0.013266 seconds.


In [41]:
clf.score(X_test, y_test)

0.86799999999999999

In [42]:
from sklearn import neighbors

max_neighbors=100
inc_by = 3
# starting from 2 - 20 neighbors went approximately linearly from 0.83 to 0.877
num_neighbors = range(3,max_neighbors,inc_by)
accuracies = []
for k in num_neighbors:
    clf = neighbors.KNeighborsClassifier(k)#, weights='distance')
    clf.fit(X_train, y_train)
    start_fit = time.clock()
    score = clf.score(X_test, y_test)
    finish_fit = time.clock()
    accuracies.append(score)    
    print 'Accuracy for ' + str(k) + ' neighbors: ' + str(score)
    print 'Time spent calculating neighbors: ' + str(finish_fit - start_fit)
    print ''

Accuracy for 3 neighbors: 0.888
Time spent calculating neighbors: 1.007553

Accuracy for 6 neighbors: 0.885
Time spent calculating neighbors: 1.005407

Accuracy for 9 neighbors: 0.897
Time spent calculating neighbors: 1.142575

Accuracy for 12 neighbors: 0.896
Time spent calculating neighbors: 1.104885

Accuracy for 15 neighbors: 0.895
Time spent calculating neighbors: 1.231566

Accuracy for 18 neighbors: 0.897
Time spent calculating neighbors: 1.217681

Accuracy for 21 neighbors: 0.893
Time spent calculating neighbors: 1.257032

Accuracy for 24 neighbors: 0.893
Time spent calculating neighbors: 1.218211

Accuracy for 27 neighbors: 0.888
Time spent calculating neighbors: 1.229593

Accuracy for 30 neighbors: 0.886
Time spent calculating neighbors: 1.228119

Accuracy for 33 neighbors: 0.886
Time spent calculating neighbors: 1.279681

Accuracy for 36 neighbors: 0.886
Time spent calculating neighbors: 1.287062

Accuracy for 39 neighbors: 0.881
Time spent calculating neighbors: 1.256163

Ac